ref: https://github.com/onnx/tutorials/blob/master/tutorials/PytorchOnnxExport.ipynb

tensorflow backend
- alexnet ...X
- vgg11 ...O
- vgg11_bn...O
- densenet121 ..X
- inception_v3 ..X
- resnet18 ..X
- squeezenet1_0 ..X

# Save pre-trained model to ONNX

In [1]:
from torch.autograd import Variable
import torch.onnx
import torchvision

In [2]:
# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.
dummy_input = Variable(torch.randn(1, 3, 224, 224))
# Obtain your model, it can be also constructed in your script explicitly
model = torchvision.models.vgg19(pretrained=True)
# Invoke export
torch.onnx.export(model, dummy_input, "vgg19.onnx")

# Load model from ONNX

In [3]:
import onnx

# Load the ONNX model
model = onnx.load("vgg19.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %1[FLOAT, 1x3x224x224]
) initializers (
  %2[FLOAT, 64x3x3x3]
  %3[FLOAT, 64]
  %4[FLOAT, 64x64x3x3]
  %5[FLOAT, 64]
  %6[FLOAT, 128x64x3x3]
  %7[FLOAT, 128]
  %8[FLOAT, 128x128x3x3]
  %9[FLOAT, 128]
  %10[FLOAT, 256x128x3x3]
  %11[FLOAT, 256]
  %12[FLOAT, 256x256x3x3]
  %13[FLOAT, 256]
  %14[FLOAT, 256x256x3x3]
  %15[FLOAT, 256]
  %16[FLOAT, 256x256x3x3]
  %17[FLOAT, 256]
  %18[FLOAT, 512x256x3x3]
  %19[FLOAT, 512]
  %20[FLOAT, 512x512x3x3]
  %21[FLOAT, 512]
  %22[FLOAT, 512x512x3x3]
  %23[FLOAT, 512]
  %24[FLOAT, 512x512x3x3]
  %25[FLOAT, 512]
  %26[FLOAT, 512x512x3x3]
  %27[FLOAT, 512]
  %28[FLOAT, 512x512x3x3]
  %29[FLOAT, 512]
  %30[FLOAT, 512x512x3x3]
  %31[FLOAT, 512]
  %32[FLOAT, 512x512x3x3]
  %33[FLOAT, 512]
  %34[FLOAT, 4096x25088]
  %35[FLOAT, 4096]
  %36[FLOAT, 4096x4096]
  %37[FLOAT, 4096]
  %38[FLOAT, 1000x4096]
  %39[FLOAT, 1000]
) {
  %41 = Conv[dilations = [1, 1], group = 1, kernel_shape = [3, 3], pads = [1, 1, 1, 1], strides = [1, 1]](%1, %

# Inference with your prefered toolkit

In [4]:
from PIL import Image 
import numpy as np
imgfile = './data/cat.jpeg'
img = Image.open(imgfile).resize( (224, 224) )
img_arr = np.array(img)
img_arr = np.expand_dims(img_arr, -1)
img_arr = np.transpose(img_arr, (3,2,0,1))/255.0
img_arr.shape

(1, 3, 224, 224)

In [5]:
import onnx_tf.backend as backend

rep = backend.prepare(model, device="CUDA:0") # or "CPU"


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python2.7/dist-packages/onnx_tf/backend.py:677: UserWarning: Unsupported kernel_shape attribute by Tensorflow in Conv operator. The attribute will be ignored.
  UserWarning)


In [6]:
outputs = rep.run(img_arr.astype(np.float32))

In [7]:
outputs = np.array(outputs).squeeze(0)
print(outputs.shape)

(1, 1000)


In [8]:
outputs = np.array(outputs).squeeze(0)
print(outputs.shape)

(1000,)


In [9]:
from imagenet1000_clsid_to_human import cls_dict

In [10]:
str_ = ""
for i in reversed(np.argsort(outputs)[-5:]):
    str_ +=  "{:.2f}% : {} \n".format(outputs[i], cls_dict[i])

In [11]:
print (str_)

13.23% : Egyptian cat 
10.91% : tabby, tabby cat 
10.35% : tiger cat 
7.97% : lynx, catamount 
7.57% : vase 

